In [24]:
import pandas as pd

In [25]:
df_shelter_effect = pd.read_excel("https://www.insee.fr/fr/statistiques/fichier/6524161/ia75.xlsx",
                                  sheet_name="Figure 3", skiprows=3, skipfooter=7)

In [26]:

df_shelter_effect = df_shelter_effect.rename(columns={'Unnamed: 0': 'long_coicop',
                                                      'Effet total.1': 'inflation_with_shelter', 'Effet total': 'inflation_without_shelter'})
df_shelter_effect = df_shelter_effect.set_index('long_coicop')

In [27]:
# Create an empty DataFrame to store the removed rows
df_other = pd.DataFrame(columns=df_shelter_effect.columns)

# Remove and move the two rows
for index in ["  électricité, gaz et autres combustibles", "  dépenses d'utilisation de véhicules"]:
    # Add the row to other_df
    df_other = pd.concat([df_other, df_shelter_effect.loc[[index]]], axis=0)
    df_shelter_effect = df_shelter_effect.drop(index)

In [28]:
# Create a list of values for the "coicop" column
coicop_values = ['{:02d}'.format(i)
                 for i in range(1, len(df_shelter_effect)+1)]

# Add the "coicop" column to the DataFrame
df_shelter_effect['coicop'] = coicop_values

In [29]:
df_other = df_other.assign(coicop=["045", "072"])

In [30]:
df_shelter_effect = pd.concat([df_other, df_shelter_effect], axis=0)

In [31]:
df_shelter_effect['shelter_effect'] = df_shelter_effect['inflation_with_shelter'] - \
    df_shelter_effect['inflation_without_shelter']

In [32]:

df_shelter_effect = df_shelter_effect.drop(['Unnamed: 1', 'Effet direct', 'Effet indirect',
                                            'Effet direct.1', 'Effet indirect.1'], axis=1)

In [33]:
df_shelter_effect = df_shelter_effect.set_index('coicop')

In [34]:

df_shelter_effect = df_shelter_effect.drop(
    ["inflation_without_shelter", "inflation_with_shelter"], axis=1)

In [35]:
import pandas as pd


url = "https://www.insee.fr/fr/statistiques/fichier/4648335/TF106.csv"

df_prop = pd.read_csv(url, sep=";")

discriminatingColumn = "DECUC"

In [36]:
df_prop_other = df_prop[df_prop["NOMENCLATURE"].isin(["045", "072"])]

In [37]:
# filtering values with exactly 2 characters, corresponding to ECOICOP classes
df_prop = df_prop.loc[df_prop['NOMENCLATURE'].str.len() == 2]

In [38]:
df_prop = pd.concat([df_prop_other, df_prop], axis=0)

In [39]:
# renaming the "NOMENCLATURE" column to "coicop"
df_prop = df_prop.rename(columns={'NOMENCLATURE': 'coicop'})

In [40]:
# filtering out rows where the "coicop" value starts with "13"
df_prop = df_prop.loc[~df_prop['coicop'].str.startswith('13')]

In [41]:
df_add04x = df_prop.loc[df_prop['coicop'] == '04'].copy()
df_add04x['coicop'] = '04x'
df_add04x['CONSO'] = df_prop.loc[df_prop['coicop'] == '04', 'CONSO'].values - \
    df_prop.loc[df_prop['coicop'] == '045', 'CONSO'].values

In [42]:
df_add07x = df_prop.loc[df_prop['coicop'] == '07'].copy()
df_add07x['coicop'] = '07x'
df_add07x['CONSO'] = df_prop.loc[df_prop['coicop'] == '07',
                                 'CONSO'].values - df_prop.loc[df_prop['coicop'] == '072', 'CONSO'].values

In [43]:
df_prop = pd.concat([df_prop, df_add04x, df_add07x],
                    ignore_index=True, sort=False)

In [44]:
df_prop.loc[(df_prop[discriminatingColumn] == 'TOT') & (
    df_prop['coicop'] == '045'), 'CONSO'].values[0]

1353

In [45]:
prop045 = 52.03/323.9
prop072 = 99.4/170

In [46]:
df_prop = df_prop[~df_prop['coicop'].isin(['04', '07'])]

In [47]:
# calculating the sum of "CONSO" for each unique value in "DECUC" column
sum_by_decuc = df_prop.groupby(discriminatingColumn)['CONSO'].sum()

In [48]:
# adding a new column "PROP" with the percentage of each row relative to the sum for each unique value in "DECUC" column
df_prop['PROP'] = df_prop['CONSO'] / \
    df_prop[discriminatingColumn].map(sum_by_decuc)

In [58]:
shelter_effect_04x = (df_shelter_effect.loc["04", "shelter_effect"] -
                      df_shelter_effect.loc["045", "shelter_effect"]*prop045)/(1-prop045)
shelter_effect_07x = (df_shelter_effect.loc["07", "shelter_effect"] -
                      df_shelter_effect.loc["072", "shelter_effect"]*prop072)/(1-prop072)

In [61]:
df_shelter_effect.loc["04x"] = shelter_effect_04x
df_shelter_effect.loc["07x"] = shelter_effect_07x

In [62]:
df_prop = df_prop.merge(right=df_shelter_effect, on="coicop")

In [63]:
df_prop["inflation_avoided"] = df_prop["PROP"]*df_prop["shelter_effect"]